# 1、训练中文tokenizer

## 1.1 加载数据集

In [3]:
from datasets import load_dataset

data = load_dataset('/Users/zhangjialin/Documents/项目/DataSets/shibing624/medical', name='pretrain')
data

HF google storage unreachable. Downloading and preparing it from source


KeyboardInterrupt: 

## 1.3 查看数据集

In [7]:
for data_type in ['train', 'validation', 'test']:
  for id, item in enumerate(data[data_type]):
    if id > 5:
      break
    print(f"{data_type}: {item}")

train: {'text': '毒蕈中毒预防是什么？1.切勿采摘自己不认识的蘑菇食用。2.毫无识别毒蕈经验者，千万不要自采蘑菇。3.预防措施：加强宣传、避免误食。4.有毒野生菇（菌）类常具备以下特征：1）色泽鲜艳度高。2）伞形等菇（菌）表面呈鱼鳞状。3）菇柄上有环状突起物。4）菇柄底部有不规则突起物。5）野生菇（菌）采下或受损，其受损部流出乳汁。'}
train: {'text': '孕妇得了霉菌性阴道炎不治疗会影响胎儿吗？在目前很多的孕妇并没有能够顺利度过孕期的，在怀孕期间多少都会遇到一点炎症问题，比如说外阴炎、阴道炎，炎症影响都比较大。一般最常见的并没有单纯性的炎症，最普通的就是霉菌性阴道炎，这是一种常见的炎症疾病，会严重影响到胎儿的身体健康，那么孕妇得了霉菌性阴道炎不治疗会影响胎儿吗，我给大家详细的介绍下。1、虽然在目前的医学上面，还没有听说哪个孕妇是因为阴道炎导致肚子里面的宝宝不能出生的，但实际上阴道炎多少都会有影响，毕竟这是炎症疾病，孕妇患阴道炎确实会影响到宝宝的身体健康，甚至有可能会导致宝宝的身体也会被感染。2、出现了霉菌性阴道炎以后要注意，比较严重的，最好是不能顺产，需要刨腹产才可以恢复，因为只有这样才能防止在分娩的时候，宝宝需要经过阴道，这样就会让霉菌穿透孩子的身体，宝宝的皮肤本身就很幼嫩，还没有发育完善，这个时候就会导致孩子也会感染霉菌。3、孕妇在怀孕期间出现阴道炎并不是不能治疗的，但是也需要控制，不过不能用药治疗，因为随便用药会伤害身体，偏方也最好是不要用，因为有的偏方可能会导致流产，每天可以用温水把外阴冲洗干净，每周三次用小苏打清洗阴道，帮助阴道改变环境。怀孕期间要注意卫生，不管是有没有出现炎症，都应该要注意内裤穿纯棉的，内裤每天都要更换，每个星期都应该把内裤消毒，平时内裤穿三个月就不能再穿了，特别是孕妇怀孕期间要注意，没有分泌物，可以分泌物过多到医院去检查。最后我们要说的是生活中孕妇应该重视起来，患上此病后，应该科学合理的进行治疗，同时也要注意好的心态，避免导致孩子的成长环境受到影响。'}
train: {'text': '卵巢炎的初期症状是什么？导致女性出现性交痛：卵巢炎由于受到盆腔炎的影响，就会使夫妻双方在性交时出现疼痛，而这将会影响到夫妻双方的性生活，严重的还将会导致夫妻双方的感情发生破裂。腹痛：多数卵巢炎患者下腹有不同程度疼痛，多为

## 1.4 安装transformers和sentencepiece

In [10]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00


## 1.5 训练中文医疗tokenizer

In [ ]:
import sentencepiece as spm

# 构建训练数据
with open('./train.txt', 'w', encoding='utf-8') as fo:
  for data_type in ['train', 'validation', 'test']:
    for item in data[data_type]:
      fo.write(item['text'].strip())
      fo.write('\n')
# 开始训练
train_config = '--input=./train.txt --model_prefix=medical_zh\
                --vocab_size=30000 --character_coverage=0.9995\
                model_type=bpe'
spm.SentencePieceTrainer.Train(train_config)

## 1.6 合并词表

In [ ]:
from transformers import LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_ob2_model

# 加载llama2的分词模型
llama_tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-70b-hf')
llama_spm = sp_ob2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())

# 加载中文医疗分词模型
medical_spm_model = spm.SentencePieceProcessor()
medical_spm_model.load('./medical_zh.model')
medical_spm = sp_ob2_model.ModelProto()
medical_spm.ParseFromString(medical_spm_model.serialized_model_proto())

# 比较llama2和中文医疗分词模型 token数
print(f"llama2 token: {len(llama_tokenizer)}, 中文医疗 token: {len(medical_spm_model)}")
print(f"llama2 all special token: {llama_tokenizer.all_special_tokens}")
print(f"llama2 allspecial id: {llama_tokenizer.all_special_ids}")
print(f"llama2 special tokens map: {llama_tokenizer.special_tokens_map}")

# 将中文医疗分词器token加入到llama2 tokenizer
llama_spm_tokens_set = set(p.piece for p in llama_spm.pieces)
for p in medical_spm.pieces:
    piece = p.piece
    if piece not in llama_spm_tokens_set:
        new_p = sp_ob2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = 0
        llama_spm.pieces.append(new_p)
# 保存
with open('./medical_llama.model', 'wb') as f:
    f.write(llama_spm.SerializeToString())
tokenizer = LlamaTokenizer(vocab_file='./medical_llama.model')
tokenizer.save_pretrained('./')